In [ ]:
!pip install ultralytics opencv-python-headless


In [ ]:
import cv2
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
from google.colab import files
import os
import time
import numpy as np


uploaded = files.upload()
video_path = list(uploaded.keys())[0]


model = YOLO('yolov8s.pt')


cap = cv2.VideoCapture(video_path)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)


fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out    = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break


    start_time = time.time()
    results     = model(frame, verbose=False)[0]
    annotated   = results.plot()
    total_time  = (time.time() - start_time) * 1000  # ms
    est_fps     = 1000 / total_time if total_time > 0 else 0


    labels            = results.names
    detected_classes  = [labels[int(c)] for c in results.boxes.cls]
    counts            = {cls: detected_classes.count(cls) for cls in set(detected_classes)}
    detection_lines   = '\n'.join([f"  - {cnt} {cls}(s)" for cls, cnt in counts.items()])


    confs    = results.boxes.conf.cpu().numpy()
    avg_conf = np.mean(confs) if len(confs) else 0.0


    print(f"Frame {frame_count}:")
    print(f"Resolution: {width}x{height}")
    print("Detected Objects:")
    print(detection_lines if detection_lines else "  - None")
    print(f"Average Confidence: {avg_conf:.2f}")
    print(f"Processing Time: {total_time:.1f}ms")
    print(f"Estimated FPS: ~{est_fps:.1f} FPS")
    print("=" * 40)


    out.write(annotated)


    if frame_count < 10:
        cv2_imshow(annotated)

    frame_count += 1


cap.release()
out.release()
cv2.destroyAllWindows()

print("✅ Done! Annotated video saved as 'output.mp4'")
